In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display



In [2]:
load_dotenv('OPENAI_API_KEY')
api_key=os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-4o-mini'

In [3]:
openai = OpenAI()

In [4]:

headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}




class website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        
        # Handle title
        self.title = soup.title.string if soup.title else "no title found"
        
        # Handle body text
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        # Handle links
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
        # Handle about section - with error handling
        about_element = soup.find('p', class_=['f4', 'my-3'])
        self.about = about_element.get_text(separator="\n", strip=True) if about_element else ""
        
        # Handle readme - with error handling
        readme_element = soup.find('article', class_=['markdown-body', 'entry-content container-lg'])
        self.readme = readme_element.get_text(separator="\n", strip=True) if readme_element else ""
        
        # Handle project title - with error handling
        project_title_element = soup.find('a', class_=['d-block', 'overflow-x-hidden', 'color-fg-default'])
        self.project_title = project_title_element.text if project_title_element else ""

        name=soup.find('span', attrs={'itemprop': 'name'})
        self.name=name.text.strip() if name else ""

    def get_contents(self):
        return f"Webpage Title:\n{self.name}\nWebpage Contents:\n{self.text}\n\n"

    def get_project_details(self):
        return f"\nAbout:\n{self.about}\n\nReadmefile content:\n{self.readme}\n\n"

In [5]:
link_system_prompt = "You are provided with a list of links found on a GitHub profile page. \
You are able to identify which links are repositories/projects created by the user, \
excluding forked repositories or links to other GitHub features.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
"repositories": [
    {"type": "project name", "url": "https://github.com/username/project-name"},
    {"type": "another project", "url": "https://github.com/username/another-project"}
]
}
"""

print(link_system_prompt)


You are provided with a list of links found on a GitHub profile page. You are able to identify which links are repositories/projects created by the user, excluding forked repositories or links to other GitHub features.
You should respond in JSON as in this example:
{
"repositories": [
    {"type": "project name", "url": "https://github.com/username/project-name"},
    {"type": "another project", "url": "https://github.com/username/another-project"}
]
}



In [6]:
def link_user_prompt(website):
    user_prompt = f"Here is the list of links from the GitHub profile {website.url} - "
    user_prompt += "please identify which repositories/projects were created by this user. \
    Exclude forked repositories, social media links, following/follower links, and other GitHub navigation links. \
    Respond with the full https URL in JSON format.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
def get_links(url):
    Website=website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":link_user_prompt(Website)}
        ],
        response_format={"type": "json_object"}
    )
    result=response.choices[0].message.content

    return json.loads(result)

In [8]:
def get_all_details(url):
    data=website(url)
    result="Github Profile Page:\n"
    result+=data.get_contents()
    result+="\n\nName of User:\n\n"
    result+=data.name
    links=get_links(url)
    # print("Found projects:", links)

    result+="\n\n Project lists:\n\n"
    for link in links["repositories"]:
        result += f"\n\n{link['type']}\n"
        result += website(link["url"]).get_project_details()
    return result


In [9]:
system_prompt = """You are a professional portfolio creation assistant that analyzes GitHub profile 'About' sections \
and README.md files to craft compelling developer portfolios. Your task is to examine:

- Profile bio and personal description
- Highlighted skills and technologies
- Featured repositories and pinned projects
- Professional goals and interests
- README content including personal introductions
- Contact information and professional links

Create a polished portfolio in markdown that effectively presents:
1. Professional summary and career objectives
2. Core technical competencies and specializations
3. Notable projects and achievements
4. Professional interests and development focus
5. Professional contact methods and social links

Structure the content to create an engaging narrative that resonates with technical recruiters \
while maintaining the developer's authentic voice and professional brand."""

In [10]:
def portfolio_user_prompt(url):
    user_prompt = f"You are analyzing {website(url).name}'s GitHub profile\n"
    user_prompt += "Based on their About section and README.md content of projects, create a professional portfolio in markdown that showcases their expertise.\n"
    user_prompt += "Here is their profile information along with project details:\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:15_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [11]:
def create_portfolio(url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": portfolio_user_prompt(url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [12]:
create_portfolio("https://github.com/karpathy")

# Andrej's Developer Portfolio

## 👨‍💻 Professional Summary

I'm Andrej, a passionate AI researcher and developer specializing in deep learning and neural networks. With a strong foundation in computer science from Stanford, I focus on creating efficient implementations of cutting-edge algorithms using C, CUDA, and Python. My goal is to push the boundaries of machine learning, enabling developers and researchers to leverage powerful tools for training and deploying large language models.

## 🚀 Career Objectives

I aim to further my expertise in deep learning, particularly in the development of scalable machine learning frameworks and optimization of neural network training processes. I seek a dynamic role in a tech company where I can contribute to innovative projects, collaborate with talented professionals, and continue learning in a challenging environment.

## 💻 Core Technical Competencies

- **Programming Languages**: C, C++, Python, JavaScript
- **Frameworks & Libraries**: CUDA, PyTorch, Jupyter Notebook
- **Machine Learning & AI**: Deep Learning, Convolutional Neural Networks (CNNs), Natural Language Processing (NLP)
- **Software Development**: Version Control (Git), Continuous Integration/Continuous Deployment (CI/CD)
- **Tools & Technologies**: Docker, TensorFlow, OpenMPI, NCCL

## 🌟 Notable Projects & Achievements

### 1. [llm.c](https://github.com/karpathy/llm.c)
A lightweight library for training large language models in pure C/CUDA. This project focuses on reproducing the GPT-2 and GPT-3 models with minimal dependencies, achieving faster training times compared to some mainstream frameworks by optimizing performance with low-level implementations.

### 2. [nanoGPT](https://github.com/karpathy/nanoGPT)
The simplest, fastest repository for training and fine-tuning medium-sized GPT models. This project emphasizes ease of use and educational content, providing clear examples and structured guidance for aspiring researchers.

### 3. [micrograd](https://github.com/karpathy/micrograd)
A minimalistic scalar-valued autograd engine that serves as a foundational component for building neural networks. This project showcases my capability to construct fundamental tools that underlie more complex AI applications.

### 4. [convnetjs](https://github.com/karpathy/convnetjs)
A JavaScript library for running deep learning models in the browser. This project democratizes access to deep learning by allowing developers to carry out computations directly in web applications.

### 5. [cryptos](https://github.com/karpathy/cryptos)
A pure Python implementation of Bitcoin for educational purposes. This project illustrates my desire to simplify complex concepts to aid learning within the open-source community.

## 📈 Professional Interests & Development Focus

My primary interests lie in the fields of:
- **Deep Learning**: Experimenting with novel architectures and large models.
- **Optimizations**: Enhancing computational efficiency of existing algorithms.
- **Education**: Developing resources and tutorials to assist newcomers to machine learning.
- **Open Source Contributions**: Supporting and contributing to diverse projects in the AI and software engineering ecosystem.

## ✉️ Professional Contact Methods

- **GitHub**: [github.com/karpathy](https://github.com/karpathy)
- **Twitter**: [@karpathy](https://twitter.com/karpathy)
- **Email**: [andrej@example.com](mailto:andrej@example.com) *(replace with actual email if needed)*

---

Thank you for viewing my portfolio! I'm excited about the potential for collaboration and am eager to contribute to innovative projects in deep learning and artificial intelligence. Let's connect!